In [5]:
import jax
import jax.numpy as jnp

from jax import grad

In [1]:
import alpa



/home/ainet/anaconda3/envs/alpa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-02 19:00:42,449	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
alpa.init(cluster="ray")

2024-11-02 19:00:43,713	INFO worker.py:1601 -- Connecting to existing Ray cluster at address: 114.212.82.188:6379...


2024-11-02 19:00:43,727	INFO worker.py:1786 -- Connected to Ray cluster.


Initializing ray with address auto
GPUlets enabled, getting GPU info from node: 114.212.82.188

all_host_info: [{'NodeID': 'ec6088ccf0536a286559323f4b78fbcae3176196150c92cacd1cb693', 'Alive': True, 'NodeManagerAddress': '114.212.82.188', 'NodeManagerHostname': 'ubuntu', 'NodeManagerPort': 42843, 'ObjectManagerPort': 38409, 'ObjectStoreSocketName': '/data/dyf/tmp/ray/session_2024-10-30_15-51-01_026041_6869/sockets/plasma_store', 'RayletSocketName': '/data/dyf/tmp/ray/session_2024-10-30_15-51-01_026041_6869/sockets/raylet', 'MetricsExportPort': 63194, 'NodeName': '114.212.82.188', 'RuntimeEnvAgentPort': 63399, 'DeathReason': 0, 'DeathReasonMessage': '', 'alive': True, 'Resources': {'memory': 66703956583.0, 'GPU': 2.0, 'node:114.212.82.188': 1.0, 'accelerator_type:G': 1.0, 'object_store_memory': 32873124249.0, 'node:__internal_head__': 1.0, 'CPU': 40.0}, 'Labels': {'ray.io/node_id': 'ec6088ccf0536a286559323f4b78fbcae3176196150c92cacd1cb693'}}]
all_host_num_devices: [2]


AttributeError: 'list' object has no attribute 'gpulets'

In [9]:
alpa.shutdown()

In [3]:
from alpa.device_mesh import get_global_cluster

cluster = get_global_cluster()
print(cluster)
cluster.print_gpus_info()

node ec6088ccf0536a286559323f4b78fbcae3176196150c92cacd1cb693 has 2 GPUs
Physical GPU: NVIDIA GeForce RTX 3090
    GPUID: 0
    UUID: GPU-174678e7-9442-e990-ec4a-2208b906ffc6
    Memory: 24.0 GB
    Node ID: ec6088ccf0536a286559323f4b78fbcae3176196150c92cacd1cb693
    Compute Capacity: 35.6
    Number of GPUlets: 0
    GPUlets:
Physical GPU: NVIDIA GeForce RTX 3090
    GPUID: 1
    UUID: GPU-b5ef7776-5b2c-5b84-3aa9-c0b1a604e7f0
    Memory: 24.0 GB
    Node ID: ec6088ccf0536a286559323f4b78fbcae3176196150c92cacd1cb693
    Compute Capacity: 35.6
    Number of GPUlets: 0
    GPUlets:


In [4]:
for key in cluster.gpu_info_per_node.keys():
    for gpu in cluster.gpu_info_per_node[key]:
        gpu.set_gpulets()
        gpu.print_info()



Physical GPU: NVIDIA GeForce RTX 3090
    GPUID: 0
    UUID: GPU-174678e7-9442-e990-ec4a-2208b906ffc6
    Memory: 24.0 GB
    Node ID: ec6088ccf0536a286559323f4b78fbcae3176196150c92cacd1cb693
    Compute Capacity: 35.6
    Number of GPUlets: 6
    GPUlets:
        GPUlets ID: 0, Memory: 4.0 GB
        belongs to NVIDIA GeForce RTX 3090: GPU-174678e7-9442-e990-ec4a-2208b906ffc6
        GPUlets ID: 1, Memory: 4.0 GB
        belongs to NVIDIA GeForce RTX 3090: GPU-174678e7-9442-e990-ec4a-2208b906ffc6
        GPUlets ID: 2, Memory: 4.0 GB
        belongs to NVIDIA GeForce RTX 3090: GPU-174678e7-9442-e990-ec4a-2208b906ffc6
        GPUlets ID: 3, Memory: 4.0 GB
        belongs to NVIDIA GeForce RTX 3090: GPU-174678e7-9442-e990-ec4a-2208b906ffc6
        GPUlets ID: 4, Memory: 4.0 GB
        belongs to NVIDIA GeForce RTX 3090: GPU-174678e7-9442-e990-ec4a-2208b906ffc6
        GPUlets ID: 5, Memory: 4.0 GB
        belongs to NVIDIA GeForce RTX 3090: GPU-174678e7-9442-e990-ec4a-2208b906ffc6
Physi

In [ ]:
global_virtual_physical_mesh = cluster.get_virtual_physical_mesh()
global_virtual_physical_mesh.print()

In [5]:
from alpa.pipeline_parallel.stage_construction import get_submesh_choices

submesh_choices = get_submesh_choices(2, 4, "all")

submesh_choices

((1, 1), (1, 2), (1, 4), (2, 4))

In [6]:
from alpa.pipeline_parallel.stage_construction import get_sub_gpulet_mesh_choices

sub_gpulet_mesh_choices = get_sub_gpulet_mesh_choices(num_gpus=2,
                                                      num_gpulets_per_gpu=[6, 6],
                                                      space="all")
sub_gpulet_mesh_choices

((1, 1), (1, 2), (1, 4))

In [20]:
from typing import Sequence
data = [3, 4]

def func(data: Sequence[int]):
    return min(data),sum(data)

func(data)

(3, 7)

In [ ]:
class GPUlets():
    id = 0
    def __init__(self):
        self.id = GPUlets.id
        GPUlets.id += 1


class GPU():
    def __init__(self, id: int):
        self.id = id
        self.gpulets = [GPUlets() for _ in range(3)]

def func(gpu_infos: Sequence[GPU]):
    result = []
    for gpu in gpu_infos:
        result.append([gpulet.id for gpulet in gpu.gpulets])